In [65]:
import numpy as np
import pandas as pd
import os
import datetime

In [66]:
INPUT_SHAPE = (1, 256)

In [67]:
data_path = '{}\\data\\jobdata.xlsx'.format( os.path.abspath('.') )
print(data_path)

d:\Files\PDF\MTECH\Sem 3\Data\data\jobdata.xlsx


In [76]:
df = pd.read_excel(data_path)

df.head(5)

,Timestamp,First Name,Middle Name (Optional),Last Name,Gender,Age,Email,Phone,City,Highest Qualification,Passing Year,Highest Qualification %,Current Company,Total Work Experience (Years),Skills,Institution Name,Upload Resume
0,2023-12-19 10:45:43.063,Digonta,NaN,Bose,Male,26,digontabose@gmail.com,9432201183,Kolkata,M.tech(cse),2024-06-03,85.6,NaN,1.0,"Java,C,Python,Mysql,DL/ML",Narula Institute of Technology,https://drive.google.com/open?id=1eM2F1bXRu1Y-...
1,2023-12-19 21:14:04.418,Anand,NaN,Kumar,Male,25,anandghb@gmail.com,8949093403,Bangalore,BE,2021-10-06,65.0,Cyberjeet Pvt. Ltd.,1.0,"CCNA, CCNP, Palo Alto Firewall",Sambhram Institute of Technology,NaN
2,2023-12-19 21:34:47.636,Nowshad,Ahmad,Dar,Male,26,mehrandar75@gmail.com,9149791345,Srinagar,BE,2021-08-10,60.0,NaN,1.0,"Html, css, bootstrap, angular, sass, typescrip...",Sambhram Institute of technology,NaN
3,2023-12-20 15:46:13.265,Anilavo,NaN,Chatterjee,Male,25,anilavo.chatterjee53@gmail.com,9123352567,Kolkata,BTech,2021-07-10,78.5,Infosys pvt ltd,2.3,"Java,Html,Spring MVC",Narula Institute of Technology,NaN
4,2023-12-20 18:05:37.085,Abdullah,Izedin,Alkinani,Male,28,Izedin2017@gmail.com,7892500843,Med,Bachelor,2022-12-20,60.0,Freelance,1.0,"Html ,css , javascript,react js",Sambhram Institute of Technology,https://drive.google.com/open?id=1Ai3Hrm3X4_tM...


In [77]:
df.describe()

,Timestamp,Age,Phone,Passing Year,Highest Qualification %,Total Work Experience (Years)
count,5,5.000000,5.000000e+00,5,5.000000,5.000000
mean,2023-12-20 03:05:17.093399808,26.000000,8.909388e+09,2022-06-16 04:48:00,69.820000,1.260000
min,2023-12-19 10:45:43.063000,25.000000,7.892501e+09,2021-07-10 00:00:00,60.000000,1.000000
25%,2023-12-19 21:14:04.417999872,25.000000,8.949093e+09,2021-08-10 00:00:00,60.000000,1.000000
50%,2023-12-19 21:34:47.636000,26.000000,9.123353e+09,2021-10-06 00:00:00,65.000000,1.000000
75%,2023-12-20 15:46:13.264999936,26.000000,9.149791e+09,2022-12-20 00:00:00,78.500000,1.000000
max,2023-12-20 18:05:37.085000,28.000000,9.432201e+09,2024-06-03 00:00:00,85.600000,2.300000
std,NaN,1.224745,5.942525e+08,NaN,11.623769,0.581378


In [78]:
df.columns

Index(['Timestamp', 'First Name', 'Middle Name (Optional)', 'Last Name',
       'Gender', 'Age', 'Email', 'Phone', 'City', 'Highest Qualification',
       'Passing Year', 'Highest Qualification %', 'Current Company',
       'Total Work Experience (Years)', 'Skills', 'Institution Name',
       'Upload Resume'],
      dtype='object')

In [79]:
def clean_data(df):
    df.drop(columns=['Timestamp', 'First Name', 'Middle Name (Optional)', 'Last Name', 'Email', 'Phone', 'Institution Name', 'Upload Resume'], inplace=True)
    # Convert text to uppercase in column: 'Highest Qualification'
    # also get the first character and produce category of bachelor, master or other
    df['Highest Qualification'] = df['Highest Qualification'].str.slice(0,1,1).str.upper()
    # One-hot encode column: 'Highest Qualification'
    df = pd.get_dummies(df, columns=['Highest Qualification'])
    
    # Replace missing values with "Freelance" in column: 'Current Company'
    df = df.fillna({'Current Company': "Freelance"})
    df['Highest Qualification %'] = df['Highest Qualification %']/100
    # Get only years
    df['Year'] = [ datetime.datetime.date(d).year for d in df['Passing Year'] ]
    
    # drop original column
    df.drop(columns=['Passing Year'], inplace=True)
    
    # find year gap, if year is less than cur year, then find year else 0
    cur_year = datetime.datetime.today().year
    
    df['Year'] = [ float(max(0, cur_year - y)) for y in df['Year'] ]
    
    # use comapany as target for deciding person was hired or not
    
    df['output'] = [ 0 if item == 'Freelance' else 1 for item in df['Current Company'] ]
    
    df.drop(columns=['Current Company'], inplace=True)
    # Convert text to lowercase in column: 'Skills'
    df['Skills'] = df['Skills'].str.lower()
    # Replace all instances of " " with "" in column: 'Skills'
    df['Skills'] = df['Skills'].str.replace(" ", "", case=False, regex=False)
    df['skill_count'] = [ len(skills.split(',')) for skills in df['Skills'] ]
    # Multi-label encode column 'Skills' using delimiter ','
    loc_0 = df.columns.get_loc('Skills')
    df_encoded = df['Skills'].str.get_dummies(sep=',').add_prefix('skill_')
    df = pd.concat([df.iloc[:,:loc_0], df_encoded, df.iloc[:,loc_0+1:]], axis=1)
    df.drop(columns=['Gender', 'City'], inplace=True)
    # Rename column 'Total Work Experience (Years)' to 'Exp'
    df = df.rename(columns={'Total Work Experience (Years)': 'Exp'})
    # Round column 'Exp' (Number of decimals: 0)
    df = df.round({'Exp': 0})
    # above 18
    df['Age'] = abs(df['Age']-18)
    return df

df_clean = clean_data(df.copy())
df_clean.head()

,Age,Highest Qualification %,Exp,skill_angular,skill_bootstrap,skill_c,skill_ccna,skill_ccnp,skill_css,skill_dl/ml,...,skill_python,skill_reactjs,skill_sass,skill_springmvc,skill_typescript,Highest Qualification_B,Highest Qualification_M,Year,output,skill_count
0,8,0.856,1.0,0,0,1,0,0,0,1,...,1,0,0,0,0,False,True,0.0,0,5
1,7,0.650,1.0,0,0,0,1,1,0,0,...,0,0,0,0,0,True,False,2.0,1,3
2,8,0.600,1.0,1,1,0,0,0,1,0,...,0,0,1,0,1,True,False,2.0,0,9
3,7,0.785,2.0,0,0,0,0,0,0,0,...,0,0,0,1,0,True,False,2.0,1,3
4,10,0.600,1.0,0,0,0,0,0,1,0,...,0,1,0,0,0,True,False,1.0,0,4


In [80]:
import tensorflow as tf

In [ ]:
base_cnn_model = tf.keras.applications.VGG16(
        weights='imagenet', 
        input_shape=INPUT_SHAPE, 
        include_top=False
    )

model = tf.keras.models.Sequential()
model.add( base_cnn_model )
model.add( tf.keras.layers.Flatten() )
model.add( tf.keras.layers.Dropout(0.5) )
model.add( tf.keras.layers.Dense(1, activation='relu') )

model.layers[0].trainable = False

model.compile(loss='binary_crossentropy', 
    optimizer=tf.keras.optimizers.Adam(),
    metrics=['accuracy'] )

model.summary()